In [1]:
import _dunsros as DR

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from kan import *
import torch
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# условно константы
bo = 1.197                      #объемный коэффициент нефти
bw = 1                          #объемный коэффициент воды
bg = 9.1 * 1e-3                 #объемный коэффициент газа
rhow = 1 * 1e3                  #плотность воды
muw = 0.28 * 1e-3               #вязкость воды при 100 гр.Ц.
sigmaw = 58.9 * 1e-3            #коэффициент поверхностного натяжения воды при 100 гр.Ц.


# условно переменные
k_og = 178                      #газовый фактор м.**3/ст.м.**3
d = 0.1524                      #диаметр трубы
theta = 90                      #угол наклона скважины
q0 = 1.59 * 1e3                 #дебит нефти н.у.
qg0 = 283 * 1e3                 #дебит газа н.у.
rs = 50.6                       #коэффициент растворимости газа в нефти (конденсации) или коэффициент выделения газа из нефти (выкипания)
rho0 = 8 * 1e2                  #плотность нефти
rhog = 7 * 1e2                  #плотность газа
mug = 0.016 * 1e-3              #вязкость газа
muo = 0.97 * 1e-3               #вязкость нефти
epsilon = 18.288 * 1e-6         #шероховатость стенки трубы
sigmao = 8.41 * 1e-3            #коэффициент поверхностного натяжения нефти
wc = 1 - qg0 / q0 / k_og        #коэффициент обводненности


# расчетные величины (кроме обводненности, которую удобно использовать как входную переменную)
qo = q0 * (1 - wc) * bo                                 #объем нефти в пластовых условиях
ql = ((1 - wc) * bo + wc * bw) * qo                     #объем жидкости в пластовых условиях
qg = (qg0 - qo * rs * (1 - wc)) * bg                    #объем газа в пластовых условиях
rho_lrc = rho0 * (1 - wc) / bo + rhow * wc / bw         #плотность жидкой фазы
rho_grc = rhog / rs                                     #плотность газа
sigma_l = sigmao * (1 - wc) + sigmaw * wc               #коэффициент поверхностного натяжения жидкость-газ
p = 180                                                 #текущее давление (где?)
mul_rc = muo * (1 - wc)  + muw * wc                     #вязкость жидкости
Sd = np.pi * d**2 / 4                                   #площадь сечения трубы

sec_p_day = 24 * 60 * 60                                #переведем все в систему СИ
qo = qo / sec_p_day
ql = ql / sec_p_day
qg = qg / sec_p_day

args_params = {
        "d": d,                         #диаметр в метрах
        "theta_deg": theta,             #угол
        "ql_rc_m3day": ql,              #дебит жидкости
        "qg_rc_m3day": qg,              #расход газа
        "rho_lrc_kgm3": rho_lrc,        #плотность жидкой фазы
        "rho_grc_kgm3": rho_grc,        #плотность газа
        "sigma_l_nm": sigma_l,          #коэффициент поверхностного натяжения жидкость-газ
        "p": p,                         #текущее давление
        "mul_rc_cp": mul_rc,            #вязкость жидкости
}

calibr_C_grav = 1                       #коэффициент калибровки гравитационной компоненты градиента
calibr_C_fric = 1                       #коэффициент калибровки фрикционной компоненты градиента

args_grav_grad = {
        "theta_deg": theta,
        "c_calibr_grav": calibr_C_grav,
}

args_fric_grad = {
        "eps_m": epsilon,
        "mul_rc_cp": mul_rc,
        "mug_rc_cp": mug,
        "c_calibr_fric": calibr_C_fric,
        "rho_lrc_kgm3": rho_lrc,
        "rho_grc_kgm3": rho_grc,
        "sigma_l_nm": sigma_l,
}

In [68]:
# протестируем работоспособность класса DunsRos()
dr_corr = DR.DunsRos()
dr_corr.calc_params(**args_params)  #класс подразумевает предварительный рассчет параметров данным методом, иначе работать не будет
dr_corr.calc_grav(**args_grav_grad) + dr_corr.calc_fric(**args_fric_grad), dr_corr.calc_grav(**args_grav_grad), dr_corr.calc_fric(**args_fric_grad)

(4246.625856792914, 4147.389976403376, 99.23588038953858)

In [47]:
dr_corr.vsm * dr_corr.vsg * dr_corr.rho_n_kgm3 / p

5.874681149677324

In [ ]:
# создадим функцию для вычисления градиента от аргументов (для более привычной записи)
# PD_DZ = lambda args_grav_grad, args_fric_grad: dr_corr.calc_grav(**args_grav_grad) + dr_corr.calc_fric(**args_fric_grad)
# list(args_fric_grad.values())

In [13]:
# let's construct a dataset
f = lambda x: x[:,0]**2 + 0.3*x[:,1] + 0.1*x[:,2]**3 + 0.0*x[:,3]
dataset = create_dataset(f, n_var=4, device=device)

input_vars = [r'$x_'+str(i)+'$' for i in range(4)]

model = KAN(width=[4,5,1], device=device)
model.fit(dataset, steps=40, lamb=0.001);

checkpoint directory created: ./model
saving model version 0.0


| train_loss: 4.47e-03 | test_loss: 4.23e-03 | reg: 4.17e+00 | : 100%|█| 40/40 [00:09<00:00,  4.34it

saving model version 0.1
